In [1]:
import pandas as pd
from datetime import datetime

def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1gZiaXkkDin9VmEbpuuQa8Xv2Jt9MdrdL/view?usp=drive_link"
direct_link = get_direct_link(link)
wiki_data = pd.read_csv(direct_link, encoding='UTF-8')

link = "https://drive.google.com/file/d/17Lr-Z508FDCLKJACnF5tDyNlXATXl_pk/view?usp=drive_link"
direct_link = get_direct_link(link)
odds_data = pd.read_csv(direct_link, encoding='UTF-8')

wiki_data['Date'] = pd.to_datetime(wiki_data['Date'])
wiki_data['date'] = wiki_data['Date']
wiki_data['pid'] = wiki_data.groupby('wiki_title').ngroup()

duplicates = wiki_data.duplicated(subset=['pid', 'Date'])
duplicates = duplicates[duplicates].index
wiki_data = wiki_data.drop(duplicates)

wiki_data = wiki_data.set_index(['pid', 'Date']).sort_index()

In [2]:
wiki_data.sort_values(by=['pid', 'date'], inplace=True)

wiki_data['wiki_med365'] = wiki_data.groupby('pid')['wiki'].rolling(window=365, min_periods=0).median().reset_index(0, drop=True)
wiki_data['wiki_yesterday'] = wiki_data.groupby('pid')['wiki'].shift()

In [3]:
wiki_data = wiki_data.dropna(subset=['month', 'year', 'day'])

In [4]:
odds_data['date'] = pd.to_datetime(odds_data['date'], format='%Y-%m-%d')
odds_data['year'] = odds_data['date'].dt.year
odds_data['month'] = odds_data['date'].dt.month
odds_data['day'] = odds_data['date'].dt.day

wiki_data['date'] = pd.to_datetime(wiki_data['date'], format='%d/%m/%Y')
wiki_data['year'] = wiki_data['date'].dt.year
wiki_data['month'] = wiki_data['date'].dt.month
wiki_data['day'] = wiki_data['date'].dt.day 


In [5]:
wiki_columns = ['date','wiki']+(list(wiki_data.columns[7:20]))

In [7]:
count_non_null_rows = wiki_data[wiki_data[['wiki_med365', 'wiki_yesterday']].notna().all(axis=1)].shape[0]
print(count_non_null_rows)

3176150


In [8]:
merged_data = pd.DataFrame()
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    red_mask = odds_data['R_fighter'].apply(lambda x: x in names)
    temp_data = odds_data[red_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]

for column in merged_data.columns:
    if column.startswith('wiki'):
        merged_data = merged_data.rename(columns={column: column + '_r'})

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350


In [9]:
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    blue_mask = odds_data['B_fighter'].apply(lambda x: x in names)
    temp_data = odds_data[blue_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
    
for column in merged_data.columns:
    if (column.startswith('wiki')) and not column.endswith('_r'):
        merged_data = merged_data.rename(columns={column: column + '_b'})

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350


In [19]:
merged_data_r = merged_data[merged_data['wiki_b'].isnull()].reset_index()
merged_data_b = merged_data[merged_data['wiki_r'].isnull()].reset_index()

combined_df = merged_data_r.combine_first(merged_data_b)

In [21]:
combined_df.to_csv('ufc_merged.csv', index=False)